### setuptools模块
虽然官方内置distutils模块也能实现类似的功能，不过现在人们更常用的是第三方模块setuptools，其相当于distutils模块的加强版，初学者推荐就使用setuptools模块。更多内容请参看setuptools模块的 [官方文档](https://setuptools.pypa.io/en/latest/) 。

现在setuptools推荐使用`setup.cfg`来进行相关配置管理，而不是之前的 `setup.py` 里的 `setup` 函数。pypi生态圈和相关PEP规范在不断完善中，现在推荐使用 `build` 模块，运行 `python -m build` 来进行你项目的打包工作。

你首先需要新建一个 `pyproject.toml` 文件，指定本项目的安装环境，setuptools相关如下：

```text
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
```


### 小型模块项目
即使是非常小型的单python文件的模块，也推荐组织成为python的模块结构而不是通过`py_modules`来指定。

现在假定这里我们讨论的python模块名字是pyskeleton，如果该python模块很小型，则推荐采用如下结构：

```text
----pyskeleton
  --__init__.py
  --another_file.py
  
setup.cfg
pyproject.toml
```

对应的setup.cfg文件内容大体如下：

```text
[metadata]
name = pyskeleton
version = attr: pyskeleton.__version__
description = quickly create a python module, have some other good concern.
url=https://github.com/a358003542/pyskeleton
long_description = file: README.md
long_description_content_type=text/markdown

[options]
include_package_data = True
packages = pyskeleton
```

也就是直接将packages写上去即可。

**NOTICE:** packages这个关键词是setuptools模块继承自distutil模块的，**其不会进一步递归扫描该模块可能有的子模块**。也就是上面这个写法只能添加pyskeleton文件夹下的 `__init__.py` 文件和与其平行的一些python文件。如果你还希望在下面添加几个子模块，则参见下面的讨论。

### 中型模块项目
一个中等规模的python模块项目如下所示：

```text
----pyskeleton
  -- submodule1
    -- __init__.py
  -- submodule2
    -- __init__.py
  --__init__.py
  --another_file.py
  
setup.cfg
pyproject.toml
```

类似上面这样中型大小的python模块项目，如果子模块不是很多的话，那么你可以继续如下这样编写：

```
packages = 
  pyskeleton
  pyskeleton.submodule1
  pyskeleton.submodule2
```

显然这样子模块多于两个以上就显得很麻烦了，那么这时就推荐使用setuptools模块提供的find函数。

```
packages = find:
```

其会自动从根目录开始进行扫描，然后根据 `__init__.py` 是否存在来自动生成packages列表。这个时候就有一个注意事项：你的测试文件夹 tests 最好不要有 `__init__.py` 文件了，实际上利用pytest这样的测试辅助模块也是不需要 `__init__.py` 文件的。

然后有的时候你可能希望某个子模块不要添加进去了，反正有的时候就有这个需求，那么你可以加上 exclude 参数：

```
[options.packages.find]
exclude = my_python_module.backup
```
注意写法是你预期要添加进入packages列表的字段，然后排除掉。

### setup.cfg的配置
 一些metadata的填写还是很简单的，不过需要注意上面的 `attr:` 和 `file:` 写法。attr可以提取本模块的某些属性信息，而可用于提取某文件的内容。

请参阅 [官方文档的这里](https://setuptools.pypa.io/en/latest/userguide/declarative_config.html) 来详细了解之。

### pip的develop模式

本小节参考了 [这个问题](https://stackoverflow.com/questions/19048732/python-setup-py-develop-vs-install) 。

对于你自己写的包，可能需要频繁变动，最好是加载引用于本地某个文件夹，那么推荐使用pip的develop安装模式，通过这种安装模式，你修改了你的模块源码是直接生效的，因为安装过程只是提供了一个引用链接，实际还是用的你的源码这边的代码。

develop安装模式的启用是，在你的python模块文件夹下运行：

```text
pip install -e .
```

### 在pypi上传你的模块
本小节讨论的内容并不实用，推荐通过Github Action的方式来上传更新你的模块，但这块内容还是保留下来了，因为具体Github Action那边的动作细节，还是这里讨论的内容，所以这块的了解，更有助于你理解Github Action的相关脚本为什么是那么编写的。

#### 正确处理README文档

现在pypi已经支持markdow文档格式了。推荐按照官方文档 [这里](<https://packaging.python.org/guides/making-a-pypi-friendly-readme/> ) 来处理：

```text
long_description = file: README.md
long_description_content_type=text/markdown
```

注意上面配置的 `long_description_content_type` ，如果你喜欢 `reStructuredText` 格式，那么设置为 `text/x-rst` 即可。

#### 打包模块
首先推荐升级最新的setuptools，wheel和twine模块。

然后直接用下面这句：

```text
python setup.py sdist bdist_wheel
```

这样将直接dist文件夹下面生成源码tar包和wheel包。

没有`setup.py` 的项目【<u>也就是采用setup.cfg新式管理方式的项目</u>】**需要安装 `build` 模块**，然后运行 `python -m build` 。

然后推荐运行下：

```text
twine check dist/*
```

来确保你的文档格式没问题。

#### 使用twine上传

使用twine上传到pypi很简单：

```text
twine upload dist/*
```

你每次都需要输入用户名和密码，你可以安装 `keyring` 模块，然后运行：

```text
keyring set https://upload.pypi.org/legacy/ your-username
```

来本地安全保存你的用户名和密码。

### 通过Github Action自动上传你的模块
可以通过Github Action来设置你的python模块更新之后自动上传到pypi那边。

请参阅 [pypi官方文档这里](https://docs.pypi.org/trusted-publishers/adding-a-publisher/) 来了解Trusted Publisher的相关细节，这样你就可以不用输入API_KEY，也不需要输入用户名和密码来上传了。

### pypi下载使用国内源
豆瓣的pypi源 `https://pypi.douban.com/simple`  或者 清华的pypi源 `https://pypi.tuna.tsinghua.edu.cn/simple` 都可以吧。

临时使用用 `-i` 或者 `--index` 选项： 

```text
pip install -i https://pypi.tuna.tsinghua.edu.cn/simple some-package
```

永久更改本地配置：

```text
pip install pip -U
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
```

### 其他注意事项
注意那个什么egg-info这个缓存文件夹，如果你更改了你的项目的配置，最好将这个缓存文件夹删掉，有时会有干扰。